### Machine example

Suppose you have 4 machines on your computer’s factory. You have to define the number of computers that each machine must produce during the next 10 hours in order to maximize the total production. The objective function of our problem is given by:
$$ max  \sum_{m \in M}\sum_{t \in T}x_{m,t}$$

where $𝑥_{𝑚,𝑡}$ represents the number of computers produced by a machine $m$ at hour $t$ Some machines have dependencies on others and the constraints are (for each $𝑡$)
$$2x_{2,t} - 8x_{3,t} \leq 0 \quad (1)$$
$$ x_{2,t} - 2x_{3,t-2} + x_{4,t} \ge 1\quad (2)$$

And the capacity production is given by (for all $t$):
$$ \sum_{m \in M}x_{m,t}\le 50 \qquad \forall t \in T \quad (3)$$
$$ 𝑥_{1,𝑡}+𝑥_{2,𝑡−1}+𝑥_{3,𝑡}+𝑥_{4,𝑡} \le 10 \quad (4)$$
$$ 0 \le x_{m,t} \le 10\quad (5)$$

Solve the problem and find the optimal production of computers for each machine for each hour of the next 10 hours
Note the $𝑥_{2,𝑡−1}$, in the last constraint, represents the second machine at hour $t-1$

Obs: Constraints (2) and (4) does not exist for t<3 and t<2, respectively


In [1]:
import pyomo.environ as pe
import pyomo.opt as po

In [2]:
m = pe.ConcreteModel()

In [3]:
# Sets
m.M = pe.RangeSet(1, 4)
m.T = pe.RangeSet(1, 10)

m.x = pe.Var(m.M, m.T, domain=pe.NonNegativeReals, bounds=(0,10))

# Constraints
def _cons1(m, t):
    return 2*m.x[2,t] - 8*m.x[3,t] <= 0
m.cons1 = pe.Constraint(m.T, rule=_cons1)

def _cons2(m, t):
    if t == 1 or t == 2:
        return pe.Constraint.Skip
    return m.x[2,t] - 2*m.x[3,t-2] + m.x[4,t] >= 1
m.cons2 = pe.Constraint(m.T, rule=_cons2)

def _cons3(m, t):
    return sum(m.x[i,t] for i in m.M) <= 50
m.cons3 = pe.Constraint(m.T, rule=_cons3)

def _cons4(m, t):
    if t == 1:
        return pe.Constraint.Skip
    return m.x[1,t] + m.x[2,t-1] + m.x[3,t] + m.x[4,t] <= 10
m.cons4 = pe.Constraint(m.T, rule=_cons4)


m.objective = pe.Objective(expr=pe.summation(m.x), sense=pe.maximize)

In [4]:
solver = po.SolverFactory('glpk')

# Gap limit
mip_gap = 0.01 # Set your desired MIP gap here (e.g., 1%)
solver.options['mipgap'] = mip_gap

# Time limit
time_limit = 50  # Set your desired time limit in seconds
solver.options['tmlim'] = time_limit


results = solver.solve(m, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --mipgap 0.01 --tmlim 50 --write C:\Users\Lenovo\AppData\Local\Temp\tmpf9uoaue_.glpk.raw
 --wglp C:\Users\Lenovo\AppData\Local\Temp\tmp8egjnn8z.glpk.glp --cpxlp C:\Users\Lenovo\AppData\Local\Temp\tmp2dfgjd4x.pyomo.lp
Reading problem data from 'C:\Users\Lenovo\AppData\Local\Temp\tmp2dfgjd4x.pyomo.lp'...
38 rows, 41 columns, 121 non-zeros
323 lines were read
Writing problem data to 'C:\Users\Lenovo\AppData\Local\Temp\tmp8egjnn8z.glpk.glp'...
321 lines were written
GLPK Simplex Optimizer, v4.65
38 rows, 41 columns, 121 non-zeros
Preprocessing...
27 rows, 38 columns, 80 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  8.000e+00  ratio =  8.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 27
      0: obj =   2.000000000e+01 inf =   8.000e+00 (8)
     16: obj =   2.800000000e+01 inf =   0.000e+00 (0)
*    32: obj =   1.281562500e+02 inf =   0.000e+00 (

In [5]:
print(pe.value(m.objective))

128.15625


In [6]:
for i in m.M:
    for t in m.T:
        if pe.value(m.x[i,t]) != 0:
            print(f"x({i}, {t}): {pe.value(m.x[i,t])}")

x(1, 1): 10.0
x(1, 5): 7.53125
x(1, 10): 7.5
x(2, 1): 10.0
x(2, 3): 9.75
x(2, 4): 1.0
x(2, 5): 5.875
x(2, 6): 7.85
x(2, 7): 8.6
x(2, 8): 4.7
x(2, 10): 10.0
x(3, 1): 8.15625
x(3, 3): 2.4375
x(3, 4): 0.25
x(3, 5): 1.46875
x(3, 6): 1.9625
x(3, 7): 2.15
x(3, 8): 1.175
x(3, 10): 2.5
x(4, 1): 10.0
x(4, 3): 7.5625
x(4, 6): 2.1625
x(4, 8): 0.224999999999999
x(4, 9): 5.3
